# **Jupyter notebook for time series forecasting based on machine learning**
### **01 Description**

This Jupyter notebook is used for traffic flow time series prediction, using the GRU model and the TGCN model (https://ieeexplore.ieee.org/abstract/document/8809901). The experimental data comes from the following related code.

### **02 Import libraries and complete instantiation**

In [1]:
# Import necessary modules
from utils.data_process import create_dataloader
from instance_parameters import Predict
from models.exp1_gru import GRU
from models.exp1_tgcn import TGCN, load_adjacency_matrix
from utils.predict import predict
from utils.test_and_inspect_fit import test, inspect_model_fit
from utils.train import train
import torch

**The parameters you can adjust** (for example, instance.model = 'GRU')
**Parameter**     | **Value**
-------- | -----
model  | 'TGCN'/'GRU'
input_size  | 40
use_gpu  | False
device  | 0
lr  | 0.001
epochs  | 100
batch_size  | 32
hidden_size  | 48
data_path  | 'dataset/filled_sz_speed.csv'
target  | '92878'
window_size  |  48
pre_len  | 4
shuffle  | True

In [2]:

# Create an instance of the Predict class
instance = Predict()
instance.model = 'TGCN'
instance.input_size = 40
instance.featuere = 'M'
instance.hidden_size = 16
instance.data_path = 'dataset/sz_speed.csv'
instance.target = '90224'
instance.epochs = 0

### **03 Confirm used device（GPU or CPU）**

In [3]:
if isinstance(instance.device, int) and instance.use_gpu:
    device = torch.device("cuda:" + f'{instance.device}')
else:
    device = torch.device("cpu")
train_loader, test_loader, valid_loader, scaler = create_dataloader(instance, device)
print("Used device:", device)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creating DataLoader<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Training set size: 2232 Test set size: 596 Validation set size: 148
Training set data:    2165 Converted to batch data: 33
Test set data:        529 Converted to batch data: 8
Validation set data:  81 Converted to batch data: 1
>>>>>>>>>>>>>>>>>>>>>>>>>>>>DataLoader Created<<<<<<<<<<<<<<<<<<<<<<<<<<<
Used device: cpu


### 03 Instantiate the model

In [4]:
try:
    print(f"Start initializing the {instance.model} model")
    if instance.model == 'GRU':
        model = GRU(input_dim = instance.input_size, hidden_size = instance.hidden_size, args = instance)
    elif instance.model == 'TGCN':
        adj = load_adjacency_matrix('dataset/sz_adj.csv')
        model = TGCN(adj = adj, hidden_size = instance.hidden_size ,args = instance)
    print(f"Initialization of the {instance.model} model is complete")
except Exception as e:
    print(f"Initialization of the {instance.model} model failed")
    print(f"Failure Details:{e}")

Start initializing the TGCN model
Initialization of the TGCN model is complete


### 04 Train model

In [5]:
if instance.train:
    print(f"Start {instance.model} model training")
    train(model, instance, train_loader, scaler)

Start TGCN model training


0it [00:00, ?it/s]

>>>>>>>>>>>>>>>>>>>>>>模型已保存,用时:0.0154 min<<<<<<<<<<<<<<<<<<


### 05 Test model

In [6]:
if instance.test:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>开始{instance.model}模型测试<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    test(model, instance, test_loader, scaler)

>>>>>>>>>>>>>>>>>>>>>>>>>开始TGCN模型测试<<<<<<<<<<<<<<<<<<<<<<<<<<<
Testset Mean Absolute Error(测试集平均绝对误差): 0.99805444
Testset Root Mean Squared Error(测试集均方根误差): 1.2683454


Target Feature Testset R2(目标特征测试集拟合曲线决定系数): -0.06467868352642703


### 06 Inspect model fitting results

In [7]:
if instance.inspect_fit:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>开始检验{instance.model}模型拟合情况<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    inspect_model_fit(model, instance, train_loader, scaler)

>>>>>>>>>>>>>>>>>>>>>>>>>开始检验TGCN模型拟合情况<<<<<<<<<<<<<<<<<<<<<<<<<<<


Target Feature Trainingset R2(目标特征训练集拟合曲线决定系数): -0.09579426092686938


### 07 Test model in another dataset

In [8]:
instance.data_path = 'dataset/los_speed.csv'
instance.target = '773869'
train_loader, test_loader, valid_loader, scaler = create_dataloader(instance, device)
if instance.test:
    print(f">>>>>>>>>>>>>>>>>>>>>>>>>开始{instance.model}模型新数据测试<<<<<<<<<<<<<<<<<<<<<<<<<<<")
    test(model, instance, test_loader, scaler)

>>>>>>>>>>>>>>>>>>>>>>>>>>>>Creating DataLoader<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
Training set size: 1512 Test set size: 404 Validation set size: 100
Training set data:    1445 Converted to batch data: 22
Test set data:        337 Converted to batch data: 5
Validation set data:  33 Converted to batch data: 0
>>>>>>>>>>>>>>>>>>>>>>>>>>>>DataLoader Created<<<<<<<<<<<<<<<<<<<<<<<<<<<
>>>>>>>>>>>>>>>>>>>>>>>>>开始TGCN模型新数据测试<<<<<<<<<<<<<<<<<<<<<<<<<<<
Testset Mean Absolute Error(测试集平均绝对误差): 0.73877484
Testset Root Mean Squared Error(测试集均方根误差): 1.1785


Target Feature Testset R2(目标特征测试集拟合曲线决定系数): -0.1757471577938272
